In [3]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaModel

In [4]:
# Mounting Google Drive to access files stored in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/NLP_Project/IMDB_Dataset_Preprocessed.csv')

In [6]:
# Load RoBERTa model and tokenizer
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = TFRobertaModel.from_pretrained("roberta-base")

# Function to get embeddings
def get_model_embeddings(tokenizer, model, review):
    inputs = tokenizer(review, return_tensors="tf", max_length=512, truncation=True, padding='max_length')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = tf.reduce_mean(last_hidden_states, axis=1)
    return sentence_embedding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [7]:
try:
  X_roberta = np.load('/content/drive/MyDrive/NLP_Project/X_roberta.npy')
  print("RoBERTa Feature Shape:", X_roberta.shape)
except:
  review  = df['cleaned_review'][0]
  X_roberta = np.array([get_model_embeddings(roberta_tokenizer, roberta_model, review)[0]])
  np.save('/content/drive/MyDrive/NLP_Project/X_roberta.npy', X_roberta)
# Convert reviews to embeddings
for i in range(X_roberta.shape[0], len(df['cleaned_review'])):
  review  = df['cleaned_review'][i]
  X_roberta = np.append(X_roberta, np.array([get_model_embeddings(roberta_tokenizer, roberta_model, review)[0]]), axis=0)
  np.save('/content/drive/MyDrive/NLP_Project/X_roberta.npy', X_roberta)

RoBERTa Feature Shape: (50000, 768)
